# IMDB 영화평 감성분석
- Pipeline
- TfidfVectorizer + LogisticRegression

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)      # 3: QUOTE_NONE
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 텍스트 전처리

In [3]:
# <br /> 태그는 공백으로 처리
df.review = df.review.str.replace('<br />',' ')

In [4]:
# 영문자 이외는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()

<ipython-input-4-b35451a0e455>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()


- Train/Test dataset으로 분리

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)

- Pipeline: TfidfVectorizer + Logistic Regression

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [8]:
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr', LogisticRegression(random_state=2022))
])

In [9]:
# 윗 셀의 코드와 동일한 코드
tvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([('tvect', tvect), ('lr', lr)])

In [10]:
# 학습
%time pipeline.fit(X_train, y_train)

Wall time: 1min 7s


Pipeline(steps=[('tvect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('lr', LogisticRegression(random_state=2022))])

In [11]:
# 평가
pipeline.score(X_test, y_test)

0.87472

- 최적 하이퍼 파라메터 찾기

In [12]:
from sklearn.model_selection import GridSearchCV
params = {
    'tvect__max_df': [100, 500],
    'lr__C': [1, 10]
}

In [13]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)

Wall time: 3min 43s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'lr__C': [1, 10], 'tvect__max_df': [100, 500]},
             scoring='accuracy')

In [14]:
grid_pipe.best_params_

{'lr__C': 10, 'tvect__max_df': 500}

In [15]:
params = {
    'tvect__max_df': [500, 1000],
    'lr__C': [10, 20]
}
%time grid_pipe.fit(X_train, y_train)

Wall time: 3min 38s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'lr__C': [1, 10], 'tvect__max_df': [100, 500]},
             scoring='accuracy')

In [16]:
grid_pipe.best_params_

{'lr__C': 10, 'tvect__max_df': 500}

In [17]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.87552

- 모델 저장

In [18]:
import joblib
joblib.dump(grid_pipe.best_estimator_, 'model/imdb_pipe.pkl')

['model/imdb_pipe.pkl']

In [19]:
best_pipe = joblib.load('model/imdb_pipe.pkl')

- 실제 데이터

In [20]:
review = '''
\nBefore all of the hype I wanted to see Spider-Man in the theater, and the hype just made me want to see it more. I didn\'t know what to expect, but I certainly expected better than what I saw. Besides being a snooze fest, it was so sappy and saccharin I scheduled a dentist visit during the movie.\n\nThe movie started going wayward the moment Peter Parker (Tom Holland) went to Doctor Strange (Benedict Cumberbatch) for a spell to make everyone forget him. It wasn\'t a bad idea for Peter to go to Doctor Strange to fix his little revealed-identity problem, it was just oddly dumb of Doctor Strange to actually accommodate Peter. Doctor Strange truly attempted to make everyone in the world forget who Peter Parker was. It was a massive spell that seemed very excessive just to fix one man\'s problems. The movie went into silly mode as Parker kept interrupting Doctor Strange as he was doing the spell. The result was a host of old nemeses from different universes converging upon this universe to find Parker. If it was explained, it was explained poorly.\n\nAt this point the movie was on shaky ground, but it hadn\'t given way. "No Way Home" would suffer another setback when Parker fought Strange to prevent him from sending all of these villains back to their respective universes. You see, if they were to go back then they\'d meet death at the hands of, or because of, that universe\'s Spider-Man, and this universe\'s Spider-Man was way too moralistic to allow that to happen. No, Parker would fix it so that they could all go back home and they could all live happily ever after.\n\nEverything after this part of the movie was immaterial to me in the grand scheme. For me, when a plot is based upon a faulty or simply bad premise, everything that follows is equally faulty and meaningless. Not that the things that follow can\'t be cool aesthetically; they\'re just empty.\n\nParker was operating upon the righteous advice of his Aunt May (Marisa Tomei). Per her wisdom, Parker had an obligation to help these poor misguided men, no matter what the exponentially more knowledgeable Doctor Strange said. Doctor Strange wanted to do the most logical thing which was send each villain back to his own universe and let fate take over from there. Parker, with his oversized heart and undersized common sense, wanted to keep them in his own universe as pet projects until he could fix them, then send them back. Nevermind that he was going to be actively tampering with the course of events of another universe thereby massively altering their timelines with untold consequences, he simply wasn\'t old enough, wise enough, or experienced enough to know what was the best thing to do. Furthermore, it was a slap in the face of the other Spider-Men. As if to say, "You guys screwed up, I\'m going to find a better, more wholesome solution because I\'m a better Spider-Man"\n\nI was so thoroughly perturbed by this new mission of Spider-Man\'s to save these villains that everything after that began to annoy me. The attempts at comedy were awkward and grossly unfunny. The emotional moments were too often and too long. The final nail in the coffin of this was the runtime. Boy did they drag this out. I could put up with a didactic and flawed plot if you get us through it in a quick and exciting way, but in their attempts to have this grand send off for Spider-Man we had to suffer through an ocean of tears, a chasm of emotions, and an abundance of silence with slow music. I left the theater drained and upset as though I\'d just been to the DMV. Everything was set up for them to reboot Spider-Man if needed, but at this point I could use a break and no more Spider-Men about "home."\n
'''
review

'\nBefore all of the hype I wanted to see Spider-Man in the theater, and the hype just made me want to see it more. I didn\'t know what to expect, but I certainly expected better than what I saw. Besides being a snooze fest, it was so sappy and saccharin I scheduled a dentist visit during the movie.\n\nThe movie started going wayward the moment Peter Parker (Tom Holland) went to Doctor Strange (Benedict Cumberbatch) for a spell to make everyone forget him. It wasn\'t a bad idea for Peter to go to Doctor Strange to fix his little revealed-identity problem, it was just oddly dumb of Doctor Strange to actually accommodate Peter. Doctor Strange truly attempted to make everyone in the world forget who Peter Parker was. It was a massive spell that seemed very excessive just to fix one man\'s problems. The movie went into silly mode as Parker kept interrupting Doctor Strange as he was doing the spell. The result was a host of old nemeses from different universes converging upon this universe 

In [21]:
# 텍스트 전처리
import re
clean_review = re.sub('[^A-Za-z]',' ',review).strip()

In [22]:
best_pipe.predict([clean_review])

array([0], dtype=int64)